In [1]:
!pip install mmcv-full
!git clone https://github.com/OCM-7898/modified_mmdetection.git
%cd modified_mmdetection
!pip install -r requirements.txt
!pip install grip

import mmdet
mmdet.__path__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.1/575.1 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for mmcv-full: filename=mmcv_full-1.6.2-cp37-cp37m-linux_x86_64.whl size=28373754 sha256=62d1088d9676bf25ff8321e97b4c9b7c81bfed829386e4835a923497c16467a1
  Stored in directory: /root/.cache/pip/wheels/51/0a/2c/990e1866b4c28e9a70ae3a0a304eeaf4711e85a87c647b8b79
Successfully built mmcv-full
Cloning into 'modified_mmdetection'...
remote: Enumerating objects: 24890, done.
remote: Total 24890 (delta 0), reused 0 (delta 0), pack-reused 24890
Receiving objects: 100% (24890/24890), 37.85 MiB | 20.22 MiB/s, done.
Resolving deltas: 100% (17468/17468), done.
/kaggle/working/modified_mmdetection
Obtaining mmtrack from git+https://github.com/open-mmlab/mmtracking#egg=mmtrack (from -r requirements/tests.txt (line 8))
  Cloning https://github.com/open-mmlab/mmtracking to ./src/mmtrack
  Running command git clone --filter=blob:none --quiet https://github.co

['/kaggle/working/modified_mmdetection/mmdet']

In [2]:
!python setup.py develop 

running develop
/opt/conda/lib/python3.7/site-packages/setuptools/command/easy_install.py:159: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  EasyInstallDeprecationWarning,
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
running egg_info
creating mmdet.egg-info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing requirements to mmdet.egg-info/requires.txt
writing top-level names to mmdet.egg-info/top_level.txt
writing manifest file 'mmdet.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'mmdet.egg-info/SOURCES.txt'
/opt/conda/lib/python3.7/site-packages/torch/utils/cpp_extension.py:387: UserWarning: Attempted to use ninja

In [3]:
from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, init_detector, inference_detector
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

import mmdet as mmdet
print(mmdet.__version__)

from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

import os
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import glob
import cv2
import shutil
import random
import os.path as osp
import json
import mmcv
import random
import re
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from typing import Dict, List
from mmdet.apis import inference_detector, init_detector, show_result_pyplot, set_random_seed

1.11.0 True
2.25.1
11.0
GCC 9.4


In [4]:
pwd

'/kaggle/working/modified_mmdetection'

In [5]:
cd ..

/kaggle/working


In [6]:
!mkdir /kaggle/working/data
!mkdir /kaggle/working/data/train-images
!mkdir /kaggle/working/data/val-images
!cp -a ../input/flowimgfixedannotations/train/train/. /kaggle/working/data/train-images
!cp -a ../input/flowimgfixedannotations/test/test/. /kaggle/working/data/val-images
!cp -a ../input/flowimgfixedannotations/test.json /kaggle/working
!cp -a ../input/flowimgfixedannotations/train.json /kaggle/working

In [55]:
cd /kaggle/working/modified_mmdetection

/kaggle/working/modified_mmdetection


In [51]:
pwd

'/kaggle/working/modified_mmdetection'

In [56]:
%%writefile configs/faster_rcnn/we_cascade_rcnn_r101_fpn.py
dataset_type = 'CocoDataset'
classes = ('bottle',)
data_root = '/kaggle/working/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
data = dict(
    samples_per_gpu=2,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        ann_file=data_root + 'train.json', # Modified
        img_prefix=data_root + 'data/train-images/images', # Modified
        classes=classes,# Added
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        ann_file=data_root + 'test.json', # Modified
        img_prefix=data_root + 'data/val-images/images', # Modified
        classes=classes, # Added
        pipeline=test_pipeline),
    test=dict(
        type=dataset_type,
        ann_file=data_root + 'test.json', # Modified
        img_prefix=data_root + 'data/val-images/images', # Modified
        classes=classes, # Added
        pipeline=test_pipeline))
evaluation = dict(interval=1, metric='bbox')
model = dict(
    type='CascadeRCNN',
    backbone=dict(
        type='ResNet',
        depth=101,
        #num_stages=4,
        #out_indices=(0, 1, 2, 3),
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet101')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        #add_extra_convs = 'on_input',  
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[.0, .0, .0, .0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
    roi_head=dict(
        type='CascadeRoIHead',
        num_stages=3,
        stage_loss_weights=[1, 0.5, 0.25],
        bbox_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        bbox_head=[
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=1,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0., 0., 0., 0.],
                    target_stds=[0.1, 0.1, 0.2, 0.2]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                loss_bbox=dict(type='SmoothL1Loss', beta=1.0,
                               loss_weight=1.0)),
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=1,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0., 0., 0., 0.],
                    target_stds=[0.05, 0.05, 0.1, 0.1]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                loss_bbox=dict(type='SmoothL1Loss', beta=1.0,
                               loss_weight=1.0)),
            dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=1,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0., 0., 0., 0.],
                    target_stds=[0.033, 0.033, 0.067, 0.067]),
                reg_class_agnostic=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0),
                loss_bbox=dict(type='SmoothL1Loss', beta=1.0, loss_weight=1.0))
        ]),
       
    # model training and testing settings
    train_cfg=dict(
        rpn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.7,
                neg_iou_thr=0.3,
                min_pos_iou=0.3,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=256,
                pos_fraction=0.5,
                neg_pos_ub=-1,
                add_gt_as_proposals=False),
            allowed_border=0,
            pos_weight=-1,
            debug=False),
        rpn_proposal=dict(
            nms_pre=2000,
            max_per_img=2000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=[
            dict(
                assigner=dict(
                    type='MaxIoUAssigner',
                    pos_iou_thr=0.5,
                    neg_iou_thr=0.5,
                    min_pos_iou=0.5,
                    match_low_quality=False,
                    ignore_iof_thr=-1),
                sampler=dict(
                    type='RandomSampler',
                    num=512,
                    pos_fraction=0.25,
                    neg_pos_ub=-1,
                    add_gt_as_proposals=True),
                mask_size=28,
                pos_weight=-1,
                debug=False),
            dict(
                assigner=dict(
                    type='MaxIoUAssigner',
                    pos_iou_thr=0.6,
                    neg_iou_thr=0.6,
                    min_pos_iou=0.6,
                    match_low_quality=False,
                    ignore_iof_thr=-1),
                sampler=dict(
                    type='RandomSampler',
                    num=512,
                    pos_fraction=0.25,
                    neg_pos_ub=-1,
                    add_gt_as_proposals=True),
                mask_size=28,
                pos_weight=-1,
                debug=False),
            dict(
                assigner=dict(
                    type='MaxIoUAssigner',
                    pos_iou_thr=0.7,
                    neg_iou_thr=0.7,
                    min_pos_iou=0.7,
                    match_low_quality=False,
                    ignore_iof_thr=-1),
                sampler=dict(
                    type='RandomSampler',
                    num=512,
                    pos_fraction=0.25,
                    neg_pos_ub=-1,
                    add_gt_as_proposals=True),
                mask_size=28,
                pos_weight=-1,
                debug=False)
        ]),
    test_cfg=dict(
        rpn=dict(
            nms_pre=1000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            score_thr=0.05,
            nms=dict(type='nms', iou_threshold=0.5),
            max_per_img=100,
            mask_thr_binary=0.5)))
optimizer = dict(type='SGD', lr=0.002, momentum=0.9, weight_decay=0.0001) # Modified
optimizer_config = dict(grad_clip=None)
# learning policy
lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    step=[8,]) # If you changed max_epochs, it's better to change when you decrease LR too here.
checkpoint_config = dict(interval=10)
# yapf:disable
log_config = dict(
    interval=50,
    hooks=[
        dict(type='TextLoggerHook'),
        # dict(type='TensorboardLoggerHook')
    ])
# yapf:enable
custom_hooks = [dict(type='NumClassCheckHook')]

dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None  # Modified
resume_from = None
runner = dict(
    type='EpochBasedRunner', # Type of runner to use (i.e. IterBasedRunner or EpochBasedRunner)
    max_epochs=30)
workflow = [('train', 1)]



Overwriting configs/faster_rcnn/we_cascade_rcnn_r101_fpn.py


In [33]:
cd ../../

/kaggle/working/modified_mmdetection


In [47]:
cd mmdet/models/necks

/kaggle/working/modified_mmdetection/mmdet/models/necks


In [41]:
%%writefile fpn_we.py
import torch.nn as nn
import torch.nn.functional as F
from mmcv.cnn import ConvModule
from mmcv.runner import BaseModule, auto_fp16
import torch
import logging
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn


import math

from ..builder import NECKS


@NECKS.register_module()
# p2, p3 in the paper is p3, p4 for us 
# format of p2, p3 is both [bs, channels, height, width]


class FPN_WE(BaseModule):
    r"""Feature Pyramid Network.
    This is an implementation of paper `Feature Pyramid Networks for Object
    Detection <https://arxiv.org/abs/1612.03144>`_.
    Args:
        in_channels (list[int]): Number of input channels per scale.
        out_channels (int): Number of output channels (used at each scale).
        num_outs (int): Number of output scales.
        start_level (int): Index of the start input backbone level used to
            build the feature pyramid. Default: 0.
        end_level (int): Index of the end input backbone level (exclusive) to
            build the feature pyramid. Default: -1, which means the last level.
        add_extra_convs (bool | str): If bool, it decides whether to add conv
            layers on top of the original feature maps. Default to False.
            If True, it is equivalent to `add_extra_convs='on_input'`.
            If str, it specifies the source feature map of the extra convs.
            Only the following options are allowed
            - 'on_input': Last feat map of neck inputs (i.e. backbone feature).
            - 'on_lateral': Last feature map after lateral convs.
            - 'on_output': The last output feature map after fpn convs.
        relu_before_extra_convs (bool): Whether to apply relu before the extra
            conv. Default: False.
        no_norm_on_lateral (bool): Whether to apply norm on lateral.
            Default: False.
        conv_cfg (dict): Config dict for convolution layer. Default: None.
        norm_cfg (dict): Config dict for normalization layer. Default: None.
        act_cfg (dict): Config dict for activation layer in ConvModule.
            Default: None.
        upsample_cfg (dict): Config dict for interpolate layer.
            Default: dict(mode='nearest').
        init_cfg (dict or list[dict], optional): Initialization config dict.
    Example:
        >>> import torch
        >>> in_channels = [2, 3, 5, 7]
        >>> scales = [340, 170, 84, 43]
        >>> inputs = [torch.rand(1, c, s, s)
        ...           for c, s in zip(in_channels, scales)]
        >>> self = FPN(in_channels, 11, len(in_channels)).eval()
        >>> outputs = self.forward(inputs)
        >>> for i in range(len(outputs)):
        ...     print(f'outputs[{i}].shape = {outputs[i].shape}')
        outputs[0].shape = torch.Size([1, 11, 340, 340])
        outputs[1].shape = torch.Size([1, 11, 170, 170])
        outputs[2].shape = torch.Size([1, 11, 84, 84])
        outputs[3].shape = torch.Size([1, 11, 43, 43])
    """

    def __init__(self,
                 in_channels,
                 out_channels,
                 num_outs,
                 start_level=0,
                 end_level=-1,
                 add_extra_convs=False,
                 relu_before_extra_convs=False,
                 no_norm_on_lateral=False,
                 conv_cfg=None,
                 norm_cfg=None,
                 act_cfg=None,
                 upsample_cfg=dict(mode='nearest'),
                 init_cfg=dict(
                     type='Xavier', layer='Conv2d', distribution='uniform')):
        super(FPN_WE, self).__init__(init_cfg)
        assert isinstance(in_channels, list)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_ins = len(in_channels)
        self.num_outs = num_outs
        self.relu_before_extra_convs = relu_before_extra_convs
        self.no_norm_on_lateral = no_norm_on_lateral
        self.fp16_enabled = False
        self.upsample_cfg = upsample_cfg.copy()

        if end_level == -1 or end_level == self.num_ins - 1:
            self.backbone_end_level = self.num_ins
            assert num_outs >= self.num_ins - start_level
        else:
            # if end_level is not the last level, no extra level is allowed
            self.backbone_end_level = end_level + 1
            assert end_level < self.num_ins
            assert num_outs == end_level - start_level + 1
        self.start_level = start_level
        self.end_level = end_level
        self.add_extra_convs = add_extra_convs
        assert isinstance(add_extra_convs, (str, bool))
        if isinstance(add_extra_convs, str):
            # Extra_convs_source choices: 'on_input', 'on_lateral', 'on_output'
            assert add_extra_convs in ('on_input', 'on_lateral', 'on_output')
        elif add_extra_convs:  # True
            self.add_extra_convs = 'on_input'

        self.lateral_convs = nn.ModuleList()
        self.fpn_convs = nn.ModuleList()

        for i in range(self.start_level, self.backbone_end_level):
            l_conv = ConvModule(
                in_channels[i],
                out_channels,
                1,
                conv_cfg=conv_cfg,
                norm_cfg=norm_cfg if not self.no_norm_on_lateral else None,
                act_cfg=act_cfg,
                inplace=False)
            fpn_conv = ConvModule(
                out_channels,
                out_channels,
                3,
                padding=1,
                conv_cfg=conv_cfg,
                norm_cfg=norm_cfg,
                act_cfg=act_cfg,
                inplace=False)

            self.lateral_convs.append(l_conv)
            self.fpn_convs.append(fpn_conv)
            
            
        self.conv_5 = ConvModule(
            256,
            256,
            kernel_size=7,
            bias=False,)
        self.conv_half = ConvModule(
            256,
            256,
            kernel_size = 1,
            bias = False,
        )
        self.conv_6 = ConvModule(
            256,
            256,
            kernel_size=5,
            bias=False,)
        self.conv_7=ConvModule(
            3*256,
            256,
            kernel_size=1,
            bias=False,)

        # add extra conv layers (e.g., RetinaNet)
        extra_levels = num_outs - self.backbone_end_level + self.start_level

        if self.add_extra_convs and extra_levels >= 1:
            for i in range(extra_levels):
                if i == 0 and self.add_extra_convs == 'on_input':
                    in_channels = self.in_channels[self.backbone_end_level - 1]

                else:
                    in_channels = out_channels
                extra_fpn_conv = ConvModule(
                    in_channels,
                    out_channels,
                    3,
                    stride=2,
                    padding=1,
                    conv_cfg=conv_cfg,
                    norm_cfg=norm_cfg,
                    act_cfg=act_cfg,
                    inplace=False)
                self.fpn_convs.append(extra_fpn_conv)

    @auto_fp16()
    def forward(self, inputs):
        """Forward function."""
        assert len(inputs) == len(self.in_channels)
        # build laterals
        laterals = [
            lateral_conv(inputs[i + self.start_level])
            for i, lateral_conv in enumerate(self.lateral_convs)
        ]
        # build top-down path
        used_backbone_levels = len(laterals)

        
        
        for i in range(used_backbone_levels - 1, 0, -1):
            # In some cases, fixing `scale factor` (e.g. 2) is preferred, but
            #  it cannot co-exist with `size` in `F.interpolate`.
            count = 0  # OUR ADDITION
            if 'scale_factor' in self.upsample_cfg:
                # fix runtime error of "+=" inplace operation in PyTorch 1.10
                print("Scale")
                laterals[i - 1] = laterals[i - 1] + F.interpolate(
                laterals[i], **self.upsample_cfg)
            else:
                #if i == 1:
                    #laterals[i-1] = FTT_get_p3pr(laterals[i],laterals[i + 1],256)

                if i == 3:
                    x = self.conv_half(laterals[i])
                    y = self.conv_5(laterals[i-1])
                    y = F.interpolate(y, size=x.shape[2:], **self.upsample_cfg)
                    z=self.conv_6(laterals[i-1])
                    z=F.interpolate(z, size=x.shape[2:], **self.upsample_cfg)
                    
                    inception = torch.cat((x,y,z),1)
                    laterals[i]=self.conv_7(inception)
#                     x = self.conv_half(laterals[i])
#                     y = self.conv_5(laterals[i-1])
#                     y = F.interpolate(y, size=x.shape[2:], **self.upsample_cfg)
#                     laterals[i] = torch.cat((x,y),1)
                if i == 2:
                    x = self.conv_half(laterals[i])
                    y = self.conv_5(laterals[i-1])
                    y = F.interpolate(y, size=x.shape[2:], **self.upsample_cfg)
                    z=self.conv_6(laterals[i-1])
                    z=F.interpolate(z, size=x.shape[2:], **self.upsample_cfg)
                    
                    inception = torch.cat((x,y,z),1)
                    laterals[i]=self.conv_7(inception)
#                     x = self.conv_half(laterals[i])
#                     y = self.conv_5(laterals[i-1])
#                     y = F.interpolate(y, size=x.shape[2:], **self.upsample_cfg)
#                     laterals[i] = torch.cat((x,y),1)
                    prev_shape = laterals[i].shape[2:]
                    laterals[i] = laterals[i] + F.interpolate(
                    laterals[i+1], size=prev_shape, **self.upsample_cfg)
                    
                if i == 1:
                    x = self.conv_half(laterals[i])
                    y = self.conv_5(laterals[i-1])
                    y = F.interpolate(y, size=x.shape[2:], **self.upsample_cfg)
                    z=self.conv_6(laterals[i-1])
                    z=F.interpolate(z, size=x.shape[2:], **self.upsample_cfg)
                    
                    inception = torch.cat((x,y,z),1)
                    laterals[i]=self.conv_7(inception)
#                     x = self.conv_half(laterals[i])
#                     y = self.conv_5(laterals[i-1])
#                     y = F.interpolate(y, size=x.shape[2:], **self.upsample_cfg)
#                     laterals[i] = torch.cat((x,y),1)
                    # for 1
                    prev_shape = laterals[i].shape[2:]
                    laterals[i] = laterals[i] + F.interpolate(
                    laterals[i+1], size=prev_shape, **self.upsample_cfg)
                    
                    prev_shape = laterals[i - 1].shape[2:]
                    laterals[i - 1] = laterals[i - 1] + F.interpolate(
                    laterals[i], size=prev_shape, **self.upsample_cfg)
                #prev_shape = laterals[i - 1].shape[2:]
                #laterals[i - 1] = laterals[i - 1] + F.interpolate(laterals[i], size=prev_shape, **self.upsample_cfg)

        
        # build outputs
        # part 1: from original levels
        outs = [
            self.fpn_convs[i](laterals[i]) for i in range(used_backbone_levels)
        ]
        
        # part 2: add extra levels
        if self.num_outs > len(outs):
            # use max pool to get more levels on top of outputs
            # (e.g., Faster R-CNN, Mask R-CNN)
            if not self.add_extra_convs:
                for i in range(self.num_outs - used_backbone_levels):
                    outs.append(F.max_pool2d(outs[-1], 1, stride=2))
            # add conv layers on top of original feature maps (RetinaNet)
            else:
                if self.add_extra_convs == 'on_input':
                    extra_source = inputs[self.backbone_end_level - 1]
                elif self.add_extra_convs == 'on_lateral':
                    extra_source = laterals[-1]
                elif self.add_extra_convs == 'on_output':
                    extra_source = outs[-1]
                else:
                    raise NotImplementedError
                outs.append(self.fpn_convs[used_backbone_levels](extra_source))
                for i in range(used_backbone_levels + 1, self.num_outs):
                    if self.relu_before_extra_convs:
                        outs.append(self.fpn_convs[i](F.relu(outs[-1])))
                    else:
                        outs.append(self.fpn_convs[i](outs[-1]))
        
        return tuple(outs)

Writing fpn_we.py


In [57]:
cd /kaggle/working/modified_mmdetection

/kaggle/working/modified_mmdetection


In [ ]:
!python tools/train.py configs/faster_rcnn/we_cascade_rcnn_r101_fpn.py

/kaggle/working/modified_mmdetection/mmdet/utils/setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/kaggle/working/modified_mmdetection/mmdet/utils/setup_env.py:49: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment variable for each process '
2022-10-21 16:31:57,959 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) [GCC 9.4.0]
CUDA available: True
GPU 0: Tesla P100-PCIE-16GB
CUDA_HOME: 

In [30]:
cd faster_rcnn

/kaggle/working/modified_mmdetection/configs/faster_rcnn


In [31]:
!python ../../tools/analysis_tools/get_flops.py 'we_cascade_rcnn_r101_fpn.py'

CascadeRCNN(
  87.919 M, 100.000% Params, 310.537 GFLOPs, 100.000% FLOPs, 
  (backbone): ResNet(
    42.275 M, 48.084% Params, 160.15 GFLOPs, 51.572% FLOPs, 
    (conv1): Conv2d(0.0 M, 0.000% Params, 2.408 GFLOPs, 0.776% FLOPs, 3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(0.0 M, 0.000% Params, 0.033 GFLOPs, 0.011% FLOPs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(0.0 M, 0.000% Params, 0.016 GFLOPs, 0.005% FLOPs, inplace=True)
    (maxpool): MaxPool2d(0.0 M, 0.000% Params, 0.016 GFLOPs, 0.005% FLOPs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      0.0 M, 0.000% Params, 13.885 GFLOPs, 4.471% FLOPs, 
      (0): Bottleneck(
        0.0 M, 0.000% Params, 4.825 GFLOPs, 1.554% FLOPs, 
        (conv1): Conv2d(0.0 M, 0.000% Params, 0.262 GFLOPs, 0.084% FLOPs, 64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(0.0 M, 0.000% Params